In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import gradio as gr
import matplotlib.pyplot as plt
import riskfolio as rp

In [2]:
## Working code

from statsmodels.tsa.statespace.sarimax import SARIMAX

def charge_data():
    data = pd.read_csv('stocks_data.csv')
    return data


def plot_portfolio(assets_list, start_year, period):
    #setting the dates
    end_year = str(int(start_year) - period)
    start_date = f'{end_year}-01-01'
    end_date = f'{start_year}-12-30'  

    # ---------- Downloading data
    df = charge_data()
    # set data index
    df = df.set_index('Date')

    # Date range
    mask = (df.index > start_date) & (df.index <= end_date)
    df = df.loc[mask]

    # Calculating  Daily returns
    Y = df[assets_list].pct_change().dropna()
    print("Y: \n",df[assets_list].head())

    # Building the portfolio object
    port = rp.Portfolio(returns=Y)
    
    # Select method and estimate input parameters:
    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.
    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)
    # Estimate optimal portfolio:
    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    #rm = 'MV' # Risk measure used, this time will be variance
    rm = 'SLPM' # Risk measure used, Second Lower Partial Moment (Sortino Ratio).
    obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'
    
    # Calculating optimal portfolio
    weights = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

    # Max Risk Adjusted Return Portfolio    
    points = 50 # Number of points of the frontier
    frontier = port.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)
    label = 'Max Risk Adjusted Return Portfolio' # Title of point
    mu = port.mu # Expected returns
    cov = port.cov # Covariance matrix
    returns = port.returns # Returns of the assets
    w=weights.reset_index()
    ######### -- plotting   --
    fig = plt.figure()
    
    
    plt.subplot(2, 2, 1)
    rp.plot_pie(w=weights, title='Sharpe Mean Variance', others=0.05, nrow=25, cmap = "tab20",
                 height=4, width=10, ax=None)
    
    plt.subplot(2, 2, 2)
    rp.plot_frontier(w_frontier=frontier, mu=mu, cov=cov, returns=returns, rm=rm,
                      rf=rf, alpha=0.05, cmap='viridis', w=weights, label=label,
                      marker='*', s=16, c='r', height=4, width=10, ax=None)


    plt.subplot(2,2,3)
    rp.plot_series(returns=Y, w=weights, cmap='tab20', height=4, width=10,
                    ax=None)

    plt.subplot(2,2,4)
    rp.plot_risk_con(w=weights, cov=cov, returns=returns, rm=rm,
                      rf=0, alpha=0.05, color="tab:blue", height=4,
                      width=10, t_factor=252, ax=None)
    return fig

def SARIMAX_predict(df, selection, pred_year):
    train = df[(df['year'] > pred_year - 3 )&(df['year'] < pred_year)]
    test = df[df['year'] == pred_year]
    my_order = (2, 2, 2)
    my_seasonal_order = (2,2,2,12)
    stickers_predict = {}
    for i in selection:
        key = f"{i}"
        train_y = train[i]
        test_y = test[i]
        SARIMAXmodel = SARIMAX(train_y, order = my_order, seasonal_order=my_seasonal_order,trend='n')
        SARIMAXmodel = SARIMAXmodel.fit()
        y_pred = SARIMAXmodel.get_forecast(len(test_y.index))
        y_pred_df = y_pred.conf_int(alpha = 0.05) 
        y_pred_df["Predictions"] = SARIMAXmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
        y_pred_df.index = test_y.index
        value = y_pred_df["Predictions"]
        stickers_predict[key] = value 
    return stickers_predict

def plot_portfolio1_predict(assets_list, start_year, period):

    # ---------- Downloading data
    df = charge_data()
    # set data index
    df = df.set_index('Date')
    pred_year=int(start_year)+1





    #setting the dates
    end_year = str(int(start_year) - period)
    start_date = f'{end_year}-01-01'
    end_date = f'{start_year}-12-30'  




    # Date range
    mask = (df.index > start_date) & (df.index <= end_date)
    df_actual = df.loc[mask]

    # Calculating  Daily returns
    Y_actual = df_actual[assets_list].pct_change().dropna()
    print("Y: \n",df[assets_list].head())

    df = charge_data()

    df.index = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df['year'] = pd.DatetimeIndex(df['Date']).year
    df['month'] = pd.DatetimeIndex(df['Date']).month
    df['weekday'] = pd.DatetimeIndex(df['Date']).weekday
    df['week'] = pd.DatetimeIndex(df['Date']).week
    
    stickers_predict = SARIMAX_predict(df,assets_list, pred_year)
    tickers_prediction_df = pd.DataFrame(stickers_predict)
    Y_predict=tickers_prediction_df.pct_change().dropna()
    
    Y_predict.index = Y_predict.index.strftime('%Y-%m-%d')


    Y_predict=Y_predict[assets_list]

    Y=pd.concat([Y_actual,Y_predict])
    print('test')
    print(Y)

    # Building the portfolio object
    port = rp.Portfolio(returns=Y)
    
    # Select method and estimate input parameters:
    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.
    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)
    # Estimate optimal portfolio:
    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    #rm = 'MV' # Risk measure used, this time will be variance
    rm = 'SLPM' # Risk measure used, Second Lower Partial Moment (Sortino Ratio).
    obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'
    
    # Calculating optimal portfolio
    weights = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

    print(weights)

    # Max Risk Adjusted Return Portfolio    
    points = 50 # Number of points of the frontier
    frontier = port.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)
    label = 'Max Risk Adjusted Return Portfolio' # Title of point
    mu = port.mu # Expected returns
    cov = port.cov # Covariance matrix
    returns = port.returns # Returns of the assets
    w=weights.reset_index()
    ######### -- plotting   --
    fig2 = plt.figure()
    plt.subplot(2, 2, 1)
    rp.plot_pie(w=weights, title='Sharpe Mean Variance', others=0.05, cmap = "tab20",
                 height=4, width=10, ax=None)
    
    plt.subplot(2, 2, 2)
    rp.plot_frontier(w_frontier=frontier, mu=mu, cov=cov, returns=returns, rm=rm,
                      rf=rf, alpha=0.05, cmap='viridis', w=weights, label=label,
                      marker='*', s=16, c='r', height=4, width=10, ax=None)


    plt.subplot(2,2,3)
    rp.plot_series(returns=Y, w=weights, cmap='tab20', height=4, width=10,
                    ax=None)

    plt.subplot(2,2,4)
    rp.plot_risk_con(w=weights, cov=cov, returns=returns, rm=rm,
                      rf=0, alpha=0.05, color="tab:blue", height=4,
                      width=10, t_factor=252, ax=None)
    print(weights)
    return fig2
   
def plot_portfolio2(assets_list, start_year, period):
    #setting the dates
    end_year = str(int(start_year)+1 - period)
    start_date = f'{end_year}-01-01'
    end_date = f'{start_year}-12-30'  

    # ---------- Downloading data
    df = charge_data()
    # set data index
    df = df.set_index('Date')

    # Date range
    mask = (df.index > start_date) & (df.index <= end_date)
    df = df.loc[mask]

    # Calculating  Daily returns
    Y = df[assets_list].pct_change().dropna()
    print("Y: \n",df[assets_list].head())

    # Building the portfolio object
    port = rp.Portfolio(returns=Y)
    
    # Select method and estimate input parameters:
    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.
    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)
    # Estimate optimal portfolio:
    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    #rm = 'MV' # Risk measure used, this time will be variance
    rm = 'SLPM' # Risk measure used, Second Lower Partial Moment (Sortino Ratio).
    obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'
    
    # Calculating optimal portfolio
    weights = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

    # Max Risk Adjusted Return Portfolio    
    points = 50 # Number of points of the frontier
    frontier = port.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)
    label = 'Max Risk Adjusted Return Portfolio' # Title of point
    mu = port.mu # Expected returns
    cov = port.cov # Covariance matrix
    returns = port.returns # Returns of the assets
    w=weights.reset_index()
    ######### -- plotting   --
    fig2 = plt.figure()
    
    
    plt.subplot(2, 2, 1)
    rp.plot_pie(w=weights, title='Sharpe Mean Variance', others=0.05, nrow=25, cmap = "tab20",
                 height=4, width=10, ax=None)
    
    plt.subplot(2, 2, 2)
    rp.plot_frontier(w_frontier=frontier, mu=mu, cov=cov, returns=returns, rm=rm,
                      rf=rf, alpha=0.05, cmap='viridis', w=weights, label=label,
                      marker='*', s=16, c='r', height=4, width=10, ax=None)


    plt.subplot(2,2,3)
    rp.plot_series(returns=Y, w=weights, cmap='tab20', height=4, width=10,
                    ax=None)

    plt.subplot(2,2,4)
    rp.plot_risk_con(w=weights, cov=cov, returns=returns, rm=rm,
                      rf=0, alpha=0.05, color="tab:blue", height=4,
                      width=10, t_factor=252, ax=None)
    return fig2

def print_input(asset, start_year, period):
    
    end_year = str(int(start_year) - period)
    start_date = f'{end_year}-01-01'
    end_date = f'{start_year}-12-30'   
    print(asset)
    print(start_date)
    print(end_date)
    return f'Your inputs are: {asset}, {start_year} and {period}' #start_year #

with gr.Blocks() as demo:
    gr.Markdown(
    """
    **Portfolio optimizer 📈 with Forecasting**: see portfolio optimization for popular market stoks 🤗
    """)
    with gr.Row():
        assets = ['JCI', 'TGT', 'CMCSA', 'CPB', 'MO', 'APA', 'MMC', 'JPM',
          'ZION', 'PSA', 'BAX', 'BMY', 'LUV', 'PCAR', 'TXT', 'TMO',
          'DE', 'MSFT', 'HPQ', 'SEE', 'VZ', 'CNP', 'NI', 'T', 'BA']
        assets.sort()
        with gr.Box():
            gr.Markdown("## Select assets to display")
            assets_list = gr.CheckboxGroup(choices=assets, label="")
    with gr.Row():
        with gr.Column():
            list = [number+2011 for number in range(2022-2011)]
            start = gr.Dropdown(list, label="Start year", value=2016)
        with gr.Column():
            period = gr.Slider(label="Number of years back for the optimization:", value=2, minimum=1, maximum=10, step=1)

    with gr.Tab("Optimizer"):
        optimize_btn = gr.Button("Optimize")
        gr.Markdown(""" Plot1: **Optimized portfolio** """)
        portfolio = gr.Plot(label="Portfolio")
        optimize_btn.click(fn=plot_portfolio, inputs=[ assets_list, start, period], outputs=portfolio)
    with gr.Tab("Forecast"):
        forecast_btn = gr.Button("Forecast")
        with gr.Column():
            gr.Markdown(""" Plot2: **Optimized portfolio for the next year forecast** """)

            plot_portfolio1 = gr.Plot(label="Portfolio1")
            forecast_btn.click(fn=plot_portfolio1_predict, inputs=[ assets_list, start, period], outputs=plot_portfolio1)
            gr.Markdown(""" Plot3: **Optimized portfolio for the next year reality** """)

            portfolio2 = gr.Plot(label="Portfolio2")
            forecast_btn.click(fn=plot_portfolio2, inputs=[ assets_list, start, period], outputs=portfolio2)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://d7963ae8524aed8f.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f318ebee5f0>,
 'http://127.0.0.1:7861/',
 'https://d7963ae8524aed8f.gradio.app')